# Project Title: Data Cleaning of Fifa 21 Dataset
# Date: March 24, 2023
# Author: Kuro Peter Doumu

## Introduction

This notebook contains cleaning up of a dataset in order to prepare it for further analysis. The purpose of this cleaning is to enable the dataset to have properly formatted columns, no input errors and clear and distinct column names. The dataset includes information about players and their main attributes.

## Data Source

The data for this project was provided by <a href="https://twitter.com/PromiseNonso_">@PromiseNonso_</a> and <a href="https://twitter.com/DataChallengeSp?t=CuCOJFQy2EjDeRDFz2D-Yw&s=09">@DataChallengeSp</a> as part of their March data challenge(obtained from kaggle), where it was downloaded and saved locally as a CSV file called "fifa21 raw data v2.csv".

#### Importing Libraries and Data

In [24]:
import pandas as pd
import numpy as np

In [25]:
path = r"C:\Users\KSD\Documents\archive\fifa21 raw data v2.csv"
df = pd.read_csv(path, low_memory=False)

In [26]:
df.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [27]:
df.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

##### Renaming Columns 

In [28]:
df = df.rename(index=str,
              columns={'↓OVA':'Overall',
                        'A/W':'Workrate_Att',
                        'D/W':'Workrate_Def',
                        'SM':'Skill_moves',
                        'W/F':'Weak_foot',
                        'IR':'Reputation',
                       
                        })

#### Looking out for Missing Values 

In [29]:
df.isna().sum().sort_values(ascending=False).head(3)

Loan Date End    17966
Hits              2595
Name                 0
dtype: int64

##### Filling the missing values in "Hits"column with 0

In [47]:
df.Hits.astype(int).fillna(0, inplace=True)
df['Hits'].isna().sum()

0

In [31]:
# Removing 'K' in Hits, changing datatype to float and multiplying by 10000
df['Hits'] = df['Hits'].str.split('K', expand=True)[0].astype(float)[69] * 1000

##### The 'Loan Date End' Column would be left as is, as most players are not on loan

#### Dealing with Data wrongly inputted

In [48]:
# df.info()

##### In the Weight column, some players weight is recorded in kg while some is in lbs


In [49]:
# These are the two types of Weight inputted in the data
df['Weight'][859:861]

859    82
860    78
Name: Weight, dtype: int64

In [34]:
# Function to remove suffixes from weight column, convert 'ibs' to kg and use apporporate datatype
def weight_convert(weights):
    if 'kg' in weights:
        return int(weights.replace('kg',''))
    else:
        return round(int(weights.replace('lbs','')) / 2.205)

In [35]:
# Applying function on Weight Column
df['Weight'] = df['Weight'].apply(weight_convert)

##### Similar to weight, the Height column has two seperate units (cm and feet)
##### We are converting to cm

In [36]:
# Looking at two seperate weight units in "Weight Column"
df['Height'][859:861]

859    178cm
860    5'11"
Name: Height, dtype: object

In [37]:
# Function to remove the suffix (cm), convert feet to cm and also use appoporaite datatype
def height_to_cm(Height):
    if 'cm' in Height:
        return int(Height.replace('cm',''))
    else:
        feet, inches = Height.split('\'')
        inches = inches.replace('"', '')
        return round((int(feet)*30.48)+(int(inches)*2.54))

In [38]:
# Applying Function on Height Column
df['Height'] = df['Height'].apply(height_to_cm)

##### The Club column has '\n\n\n\n' in front of every club name.

In [39]:
df['Club'].head()

0           \n\n\n\nFC Barcelona
1               \n\n\n\nJuventus
2        \n\n\n\nAtlético Madrid
3        \n\n\n\nManchester City
4    \n\n\n\nParis Saint-Germain
Name: Club, dtype: object

In [40]:
df['Club'] = df['Club'].str.split('\n\n\n\n', expand=True)[1]
df['Club'].head()

0           FC Barcelona
1               Juventus
2        Atlético Madrid
3        Manchester City
4    Paris Saint-Germain
Name: Club, dtype: object

##### We have to remove the '★' in the Reputation, skill_moves and weak_foot columns

In [41]:
# Looking at the columns where the '★' is present 
df[['Skill_moves', 'Weak_foot', 'Reputation']].head()

,Skill_moves,Weak_foot,Reputation
0,4★,4 ★,5 ★
1,5★,4 ★,5 ★
2,1★,3 ★,3 ★
3,4★,5 ★,4 ★
4,5★,5 ★,5 ★


In [42]:
# Function to remove '★' 
def remove_star(star):
    if '★' in star:
        amend = int(star.replace('★',''))
    return amend

In [51]:
# Applying fuction to columns
df[['Skill_moves', 'Weak_foot', 'Reputation']] = df[['Skill_moves', 'Weak_foot', 'Reputation']].applymap(remove_star)
df[['Skill_moves', 'Weak_foot', 'Reputation']].head()

,Skill_moves,Weak_foot,Reputation
0,4,4,5
1,5,4,5
2,1,3,3
3,4,5,4
4,5,5,5


#### Properly Formatting the Currency Columns ('Value', 'Wage' and 'Release Clause')

In [44]:
# The columns where 
df[['Value','Wage','Release Clause',]].head()

,Value,Wage,Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [52]:
# Creating function to convert Value, Wage and Release Clause
def convert_currency(value):
    if 'M' in value:
        fix_value = float(value.replace('€', '').replace('M','')) * 1000000
    elif 'K' in value:
        fix_value = float(value.replace('€', '').replace('K','')) * 1000
    else:
        fix_value = float(value.replace('€', ''))
    return fix_value

In [53]:
# Applying Functions on Columns
df[['Value','Wage','Release Clause',]] = df[['Value','Wage','Release Clause',]].applymap(convert_currency)

In [54]:
df[['Value','Wage','Release Clause',]].head()

,Value,Wage,Release Clause
0,103500000.0,560000.0,138400000.0
1,63000000.0,220000.0,75900000.0
2,120000000.0,125000.0,159400000.0
3,129000000.0,370000.0,161000000.0
4,132000000.0,270000.0,166500000.0


In [55]:
df['Workrate_Att'].unique()

array(['Medium', 'High', 'Low'], dtype=object)

In [56]:
df.loc[df['Workrate_Att'] == "Low", 'Workrate_Att'] = 1
df.loc[df['Workrate_Att'] == "Medium", 'Workrate_Att'] = 2
df.loc[df['Workrate_Att'] == "High", 'Workrate_Att'] = 3
df['Workrate_Att'] = df['Workrate_Att'].astype(int)

In [57]:
df['Workrate_Att'].unique()

array([2, 3, 1])

In [58]:
df['Workrate_Def'].unique()

array(['Low', 'Medium', 'High'], dtype=object)

In [59]:
df.loc[df['Workrate_Def'] == "Low", 'Workrate_Def'] = 1
df.loc[df['Workrate_Def'] == "Medium", 'Workrate_Def'] = 2
df.loc[df['Workrate_Def'] == "High", 'Workrate_Def'] = 3
df['Workrate_Def'] = df['Workrate_Def'].astype(int)

In [60]:
df['Workrate_Att'].unique()

array([2, 3, 1])

#### Properly Formatting Columns with Dates to DateTime format

In [61]:
df['Joined'] = pd.to_datetime(df['Joined'])
df['Loan Date End'] = pd.to_datetime(df['Loan Date End']) 

In [62]:
df['Joined'].head()

0   2004-07-01
1   2018-07-10
2   2014-07-16
3   2015-08-30
4   2017-08-03
Name: Joined, dtype: datetime64[ns]

#### 

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                18979 non-null  int64         
 1   Name              18979 non-null  object        
 2   LongName          18979 non-null  object        
 3   photoUrl          18979 non-null  object        
 4   playerUrl         18979 non-null  object        
 5   Nationality       18979 non-null  object        
 6   Age               18979 non-null  int64         
 7   Overall           18979 non-null  int64         
 8   POT               18979 non-null  int64         
 9   Club              18742 non-null  object        
 10  Contract          18979 non-null  object        
 11  Positions         18979 non-null  object        
 12  Height            18979 non-null  int64         
 13  Weight            18979 non-null  int64         
 14  Preferred Foot    18979 non

#### Explorting Cleaned Data to .csv

In [64]:
df.to_csv('cleaned_fifa21_data.csv')